## InfluxDB Python Client
* 메뉴얼 : http://influxdb-python.readthedocs.io/en/latest/
* 클라이언트 다운로드 : https://github.com/influxdata/influxdb-python

In [1]:
from influxdb import InfluxDBClient

In [2]:
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'telegraf')
# client.create_database('example2')

In [3]:
json_body = [
    {
        "measurement": "scom",
        "tags": {
            "host": "server01",
            "region": "us-west"
        },
        "time": "2016-11-28T10:00:00Z",
        "fields": {
            "fatal": 1,
            "critical" : 3,
            "normal" : 7,
        }
    }
]

In [4]:
client.write_points(json_body)

True

In [19]:
import argparse
import pandas as pd

from influxdb import DataFrameClient


def pandas_test(host='localhost', port=8086):
    user = 'root'
    password = 'root'
    dbname = 'example'

    client = DataFrameClient(host, port, user, password, dbname)

    print("Create pandas DataFrame")
    df = pd.DataFrame(data=list(range(100)),
                      index=pd.date_range(start='2016-08-29',
                                          periods=100, freq='H'))

    print("Write DataFrame")
    client.write_points(df, 'demo')

    print("Write DataFrame with Tags")
    client.write_points(df, 'demo', {'k1': 'v1', 'k2': 'v2'})

    print("Read DataFrame")
    client.query("select * from demo")
    
    print(df)

    
pandas_test()

Create pandas DataFrame
Write DataFrame
Write DataFrame with Tags
Read DataFrame
                            0
2016-08-29 00:00:00+00:00   0
2016-08-29 01:00:00+00:00   1
2016-08-29 02:00:00+00:00   2
2016-08-29 03:00:00+00:00   3
2016-08-29 04:00:00+00:00   4
2016-08-29 05:00:00+00:00   5
2016-08-29 06:00:00+00:00   6
2016-08-29 07:00:00+00:00   7
2016-08-29 08:00:00+00:00   8
2016-08-29 09:00:00+00:00   9
2016-08-29 10:00:00+00:00  10
2016-08-29 11:00:00+00:00  11
2016-08-29 12:00:00+00:00  12
2016-08-29 13:00:00+00:00  13
2016-08-29 14:00:00+00:00  14
2016-08-29 15:00:00+00:00  15
2016-08-29 16:00:00+00:00  16
2016-08-29 17:00:00+00:00  17
2016-08-29 18:00:00+00:00  18
2016-08-29 19:00:00+00:00  19
2016-08-29 20:00:00+00:00  20
2016-08-29 21:00:00+00:00  21
2016-08-29 22:00:00+00:00  22
2016-08-29 23:00:00+00:00  23
2016-08-30 00:00:00+00:00  24
2016-08-30 01:00:00+00:00  25
2016-08-30 02:00:00+00:00  26
2016-08-30 03:00:00+00:00  27
2016-08-30 04:00:00+00:00  28
2016-08-30 05:00:00

client.create_user('koon', 'koon', admin=True)

In [3]:
client.create_user('telegraf', 'telegraf', admin=True)

In [6]:
client.get_list_database()

[{'name': '_internal'},
 {'name': 'mydb'},
 {'name': 'example'},
 {'name': 'example2'}]

In [7]:
client.get_list_users()

[{'admin': True, 'user': 'koon'}]

In [8]:
client.query('select * from cpu_load_short', database='example')

ResultSet({'('cpu_load_short', None)': [{'host': 'server01', 'value': 0.64, 'region': 'us-west', 'time': '2009-11-10T23:00:00Z'}]})

AttributeError: 'InfluxDBClient' object has no attribute 'change_database'

In [23]:
import time
from influxdb import InfluxDBClient

class InfluxDB :
    m_conn = None
    def __init__(self) :
        pass

    def open(self) :
        bConti = True
        while bConti :
            try :
                self.m_dConn = InfluxDBClient('localhost', 8086, 'koon', 'koon', 'mydb')
                bConti = False
            except Exception:
                time.sleep(CmpGlobal.g_nConnectionRetryInterval)
                print("--------------------------------- influxdb connect fail -------------------------------------")
                
    def insertData(self, jsondata) :
        print("Write points: {0}".format(jsondata))
        self.m_dConn.write_points(jsondata)

class InfluxDBManager :
    m_oDBConn = None
    
    def __init__(self) :
        self.m_oDBConn = InfluxDB()
        self.m_oDBConn.open()

    def insert(self, nNodeId, watt, apc, pfc, pfcval, ipeak, vpeak) :
        json_meter_body = [
            {
                "measurement": "officepower",
                "tags": {
                    "node":  nNodeId
                },
                "fields": {
                    "watt":  watt,
                    "apc":  apc,
                    "pfc":  pfc,
                    "pfcval": pfcval,
                    "ipeak" : ipeak,
                    "vpeak" : vpeak
                }
            }
        ]
        self.m_oDBConn.insertData(json_meter_body)
        return 0

In [28]:
g_influxdbconn = InfluxDBManager()

# 아래와 같이 입력함.
#  g_influxdbconn.insert(nNodeId,wattVal,dWattFactor,dPowerFactor,pftVal,iPeak,vPeak)

In [29]:
g_influxdbconn.insert('Host01', 300, 20, 10, 1, 30, 20)

Write points: [{'tags': {'node': 'Host01'}, 'measurement': 'officepower', 'fields': {'pfcval': 1, 'watt': 300, 'ipeak': 30, 'pfc': 10, 'vpeak': 20, 'apc': 20}}]


0

### DataframeClient with Pandas 예제
* https://github.com/influxdata/influxdb-python/tree/master/examples

In [36]:
import argparse
import pandas as pd

from influxdb import DataFrameClient


def main(host='localhost', port=8086):
    user = 'root'
    password = 'root'
    dbname = 'example3'

    client = DataFrameClient(host, port, user, password, dbname)

    print("Create pandas DataFrame")
    df = pd.DataFrame(data=list(range(30)),
                      index=pd.date_range(start='2014-11-16',
                                          periods=30, freq='H'))

    print("Create database: " + dbname)
    client.create_database(dbname)

    print("Write DataFrame")
    client.write_points(df, 'demo')

    print("Write DataFrame with Tags")
    client.write_points(df, 'demo', {'k1': 'v1', 'k2': 'v2'})

    print("Read DataFrame")
    client.query("select * from demo")

    #print("Delete database: " + dbname)
    #client.drop_database(dbname)


def parse_args():
    parser = argparse.ArgumentParser(
        description='example code to play with InfluxDB')
    parser.add_argument('--host', type=str, required=False,
                        default='localhost',
                        help='hostname of InfluxDB http API')
    parser.add_argument('--port', type=int, required=False, default=8086,
                        help='port of InfluxDB http API')
    return parser.parse_args()

if __name__ == '__main__':
    main()
    #args = parse_args()
    #main(host=args.host, port=args.port)

Create pandas DataFrame
Create database: example3
Write DataFrame
Write DataFrame with Tags
Read DataFrame


### InfluxDB Series Helper

In [1]:
"""
Tutorial/Example how to use the class helper `SeriesHelper`
"""

from influxdb import InfluxDBClient
from influxdb import SeriesHelper

# InfluxDB connections settings
host = 'localhost'
port = 8086
user = 'root'
password = 'root'
dbname = 'test'

myclient = InfluxDBClient(host, port, user, password, dbname)

# Uncomment the following code if the database is not yet created
# myclient.create_database(dbname)
# myclient.create_retention_policy('awesome_policy', '3d', 3, default=True)


class MySeriesHelper(SeriesHelper):
    # Meta class stores time series helper configuration.
    class Meta:
        # The client should be an instance of InfluxDBClient.
        client = myclient
        # The series name must be a string. Add dependent fields/tags in curly brackets.
        server_name = 'events.stats.{server_name}'
        # Defines all the fields in this time series.
        fields = ['some_stat', 'other_stat']
        # Defines all the tags for the series.
        tags = ['server_name']
        # Defines the number of data points to store prior to writing on the wire.
        bulk_size = 5
        # autocommit must be set to True when using bulk_size
        autocommit = True


# The following will create *five* (immutable) data points.
# Since bulk_size is set to 5, upon the fifth construction call, *all* data
# points will be written on the wire via MySeriesHelper.Meta.client.
MySeriesHelper(server_name='us.east-1', some_stat=159, other_stat=10)
MySeriesHelper(server_name='us.east-1', some_stat=158, other_stat=20)
MySeriesHelper(server_name='us.east-1', some_stat=157, other_stat=30)
MySeriesHelper(server_name='us.east-1', some_stat=156, other_stat=40)
MySeriesHelper(server_name='us.east-1', some_stat=155, other_stat=50)

# To manually submit data points which are not yet written, call commit:
MySeriesHelper.commit()

# To inspect the JSON which will be written, call _json_body_():
MySeriesHelper._json_body_()

[]

In [1]:
import pandas as pd
import time

from os.path import join
from influxdb import InfluxDBClient
from influxdb import SeriesHelper

host = 'localhost'
port = 8086
user = 'root'
password = 'root'
dbname = 'test'

myclient = InfluxDBClient(host, port, user, password, dbname)

class MySeriesHelper(SeriesHelper):
    # Meta class stores time series helper configuration.
    class Meta:
        # The client should be an instance of InfluxDBClient.
        client = myclient
        # The series name must be a string. Add dependent fields/tags in curly brackets.
        series_name = '{series_name}'
        
        # Defines all the fields in this time series.
        fields = ['perf_2', 'perf_3', 'values']
        
        # Time Value
        time = ['time']
        
        # Defines all the tags for the series.
        tags = ['series_name','host']
        # Defines the number of data points to store prior to writing on the wire.
        bulk_size = 1000
        # autocommit must be set to True when using bulk_size
        autocommit = True
        
path='/Volumes/NIFTY/Bigdata/'

#날짜 컬럼 파싱
#dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
#df = pd.read_csv(infile, parse_dates=['datetime'], date_parser=dateparse)

start_time = time.time()

perf_df = pd.read_csv(join(path, 'SCOM_Perf_20160921_20161022.txt'), 
                      sep="EhdroAjd@#", names=['host', 'perf_1', 'perf_2', 'perf_3', 'values', 'regdt'], 
                      chunksize=10000, #nrows=100000, 
                      parse_dates=[5], na_values='N/A', engine='python')

#perf_df[:10]
#perf_df.info()

#perf_df['host']=perf_df['host'].str.replace('.corp.doosan.com', '',case=False)

for index, row in perf_df.iterrows():
    try:
        MySeriesHelper(series_name=row['perf_1'], host=row['host'].str.replace('.corp.doosan.com', '',case=False), 
                       perf_2=row['perf_2'], perf_3=row['perf_3'], 
                       values=row['values'], time=row['regdt'])

        MySeriesHelper.commit()
    except Exception:
        pass

print("--- %s seconds ---" % (time.time() - start_time))




AttributeError: 'TextFileReader' object has no attribute 'iterrows'

In [1]:
-


500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
8500000
9000000
9500000
10000000
10500000
11000000
11500000
12000000
12500000
13000000
13500000
14000000
14500000
15000000
15500000
16000000
16500000
17000000
17500000
18000000
18500000
19000000
19500000
20000000
20500000
21000000
21500000
22000000
22500000
23000000
23500000
24000000
24500000
25000000
25500000
26000000
26500000
27000000
27500000
28000000
28500000
29000000
29500000
30000000
30500000
31000000
31500000
32000000
32500000
33000000
33500000
34000000
34500000
35000000
35500000
36000000
36500000
37000000
37500000
38000000
38500000
39000000
39500000
40000000
40500000
41000000
41500000
42000000
42500000
43000000
43500000
44000000
44500000
45000000
45500000
46000000
46500000
47000000
47500000
48000000
48500000
49000000
49500000
50000000
50500000
51000000
51500000
52000000
52500000
53000000
53500000
54000000
54500000
55000000
55500000
56000000
56500000
570

In [50]:
from datetime import datetime

sdate = datetime.strptime("2016-10-23 21:50:00.000", "%Y-%m-%d %H:%M:%S.%f")

print(sdate)

2016-10-23 21:50:00


In [1]:
import re
p = re.compile('\d+(\.\d+)?')
strs = '---'

cflot = (0.0 if p.match(strs) == None else float(strs))

print (cflot)


0.0


In [2]:
str = '0'

float(str)

0.0

ValueError: could not convert string to float: 

In [10]:
if (10001 % 1000) == 0:
    print('a')
else :
    print('b')

b


In [11]:
float(0.0)

0.0

In [15]:
cnt = 1

while cnt < 100:
    if (cnt % 10) == 0:
        print(cnt)
    
    cnt = cnt + 1



10
20
30
40
50
60
70
80
90


In [7]:
import time
from datetime import datetime

at = datetime.strptime("2016-10-23 21:50:00.000", "%Y-%m-%d %H:%M:%S.%f")
current_time = at.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')

datetime.timestamp(at)

1477227000.0

In [5]:
from datetime import datetime 

print (datetime.fromtimestamp(1474416000))

2016-09-21 09:00:00
